# PROJET DE SOUTENANCE - DATAGONG x 10.000 Codeurs

Sujet : Prédiction du parti politique victorieux des élection présidentielles de 2020 au USA à partir de données socio-démographiques

Plan de travail :

- Setup (Installation et import des librairies nécessaires)
- Import des données
- Préparation & Constitution de la donnée exploitable
- Analyse exploratoire
- Modélisation
- Evaluation
- Conclusion

## 1. Setup

### 1.1. INSTALLATION DES DEPENDANCES

Nous devons installer les librairies suivantes :

- pandas
- numpy
- scikit-learn
- xlrd
- openpyxl
- matplotlib
- seaborn
- shap
- xgboost

Pour cela, nous pouvons utiliser la commande suivante :

```bash
pip install -r requirements.txt
```

In [1]:
# Commande pour installer les dépendances
!pip install -r requirements.txt

### 1.2. IMPORTATION DES LIBRAIRIES

In [3]:
# Importation des bibliothèques principales
  
import numpy as np  # Manipulation de tableaux numériques, gestion des calculs mathématiques  
import pandas as pd  # Gestion et manipulation des données sous forme de DataFrame  

# Visualisation des données  
import seaborn as sns  # Création de graphiques statistiques avancés  
import matplotlib.pyplot as plt  # Génération de visualisations basiques (histogrammes, scatter plots, etc.)  

# Préparation des données et séparation en train/test  
from sklearn.model_selection import train_test_split  # Découpage des données en ensembles d'entraînement et de test  
from sklearn.preprocessing import OneHotEncoder  # Encodage des variables catégorielles en variables numériques  

# Modèle de base (Régression Logistique)  
from sklearn.linear_model import LogisticRegression  # Modèle de régression logistique pour la classification  

# Évaluation des modèles  
from sklearn.metrics import classification_report  # Génération d’un rapport de performance des modèles  

# Gestion des classes déséquilibrées  
from sklearn.utils import resample  # Sous-échantillonnage de la classe majoritaire (undersampling)  
from imblearn.over_sampling import SMOTE  # Sur-échantillonnage de la classe minoritaire (oversampling)  

# Modèles avancés  
from sklearn.ensemble import RandomForestClassifier  # Modèle de Random Forest pour la classification  
from xgboost import XGBClassifier  # Modèle XGBoost pour une classification optimisée  

# Optimisation des modèles  
from sklearn.model_selection import GridSearchCV  # Recherche des meilleurs hyperparamètres avec validation croisée  
from imblearn.pipeline import Pipeline as ImbPipeline  # Création d'un pipeline intégrant le prétraitement et le modèle  

# Interprétabilité des modèles  
import shap  # Explication des prédictions des modèles avec SHAP (SHapley Additive exPlanations)  

from tqdm import tqdm  # Affichage de barres de progression pour suivre l’exécution des boucles
